In [28]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


## Retrieval Augmented Generation (RAG)

### Carregando Documentos - Loading

In [30]:
!pip install langchain_community langchain_text_splitters langchain_openai faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 29.4 MB/s eta 0:00:00


In [21]:
# https://python.langchain.com/v0.2/docs/how_to/#document-loaders
# https://python.langchain.com/v0.2/docs/integrations/document_loaders/

import bs4
from langchain_community.document_loaders import WebBaseLoader

# Filtra o conteúdo da página por uma classe específica
bs4_strainer = bs4.SoupStrainer(class_=("FragmentView_text__g6Uq2 FragmentView_verseByVerse__l1TB0"))

# Carrega o conteúdo da página
loader = WebBaseLoader(
    web_paths=("https://www.bibliaonline.com.br/ara/pv/1",),
    bs_kwargs={"parse_only": bs4_strainer},
)

# Carrega o conteúdo da página
docs = loader.load()

len(docs[0].page_content)

2841

In [22]:
print(docs[0].page_content[:])

Uso dos provérbios1 Provérbios de Salomão, filho de Davi, o rei de Israel. 2 Para aprender a sabedoria e o ensino; para entender as palavras de inteligência; 3 para obter o ensino do bom proceder, a justiça, o juízo e a equidade; 4 para dar aos simples prudência e aos jovens, conhecimento e bom siso. 5 Ouça o sábio e cresça em prudência; e o instruído adquira habilidade 6 para entender provérbios e parábolas, as palavras e enigmas dos sábios. 7 O temor do Senhor é o princípio do saber, mas os loucos desprezam a sabedoria e o ensino. Contra as seduções dos pecadores8 Filho meu, ouve o ensino de teu pai e não deixes a instrução de tua mãe. 9 Porque serão diadema de graça para a tua cabeça e colares, para o teu pescoço. 10 Filho meu, se os pecadores querem seduzir-te, não o consintas. 11 Se disserem: Vem conosco, embosquemo-nos para derramar sangue, espreitemos, ainda que sem motivo, os inocentes; 12 traguemo-los vivos, como o abismo, e inteiros, como os que descem à cova; 13 acharemos to

### Dividindo Documentos - Splitting/Chunking

In [23]:
# https://python.langchain.com/v0.2/docs/how_to/#text-splitters

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

4

In [26]:
print(all_splits[3].page_content)

porém não me hão de achar. 29 Porquanto aborreceram o conhecimento e não preferiram o temor do Senhor; 30 não quiseram o meu conselho e desprezaram toda a minha repreensão. 31 Portanto, comerão do fruto do seu procedimento e dos seus próprios conselhos se fartarão. 32 Os néscios são mortos por seu desvio, e aos loucos a sua impressão de bem-estar os leva à perdição. 33 Mas o que me der ouvidos habitará seguro, tranquilo e sem temor do mal.


### Indexando - Store

In [31]:
# https://python.langchain.com/v0.2/docs/how_to/embed_text/

from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

vectorstore = FAISS.from_documents(all_splits, OpenAIEmbeddings())

In [32]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("Como ser mais sábio?")

len(retrieved_docs)

4

### Buscando e Recuperando Informações - Retrieve

In [35]:
from langchain_core.prompts import ChatPromptTemplate

system_template = """Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.

Pergunta: {question}

Contexto: {context}

Resposta:
"""

prompt_template = ChatPromptTemplate.from_template(system_template)

In [36]:
example_messages = prompt_template.invoke({
    "context": "algum contexto",
    "question": "alguma pergunta"
})

print(example_messages.to_messages())

[HumanMessage(content='Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.\n\nPergunta: alguma pergunta\n\nContexto: algum contexto\n\nResposta:\n')]


### Gerando Respostas - Generate

In [37]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [38]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [39]:
for chunk in rag_chain.stream("como ser mais sábio?"):
    print(chunk, end="", flush=True)

Para ser mais sábio, ouça o ensino e a instrução de seus pais e busque o temor do Senhor, que é o princípio do saber. Além disso, valorize a prudência e o bom conselho, evitando a companhia de pessoas que promovem o mal.

## Exercícios

### Exercício 1
Faça um RAG com um pequeno arquivo de texto, contendo informações que, certamente, a LLM não conheça. Você deverá construir o arquivo e enviar para o ambiente de execução. Escolha a forma de chunking apropriada para o seu documento.

***R. Feito acima***